# LSTM for Smoothed Sunspot Data

#### Note:
Initially started using an RNN with a ReLu activation function, which produced some form of results, but they didn't fit well to the data.

I also attempted to use a Sigmoid function & Elu function instead, which seemed to perform worse than the Relu activation function.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import ModelCheckpoint

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [3]:
print(tf.__version__)

tf.config.list_physical_devices()

2.11.0


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
tf.config.set_visible_devices([], 'GPU')

**Note:** Unfortunately, the M1 chip's GPU runs LSTM's extrmely slowly, so I am forced to only use the CPU.

In [5]:
df = pd.read_csv('../../Data/Savitzky-Golay Data/SVG_poly1.csv') # importing data for SVG polyorder = 1
X = ((df.Year).values).reshape(-1,1)
y = (df.Sunspot_Number).values
yerr = np.zeros(len(X))

df.drop(columns=['Unnamed: 0'], inplace=True) 
# removes the unnecessary column
df.set_index("Year", inplace=True)

df.head()

,Sunspot_Number
Year,
1818.001,34.566570
1818.004,34.546053
1818.007,34.525535
1818.010,34.505017
1818.012,34.484499


## Data Preprocessing

In [6]:
limit = 65000
skips = 28 # 1 data point per luna cycle
train_data = df.iloc[:limit:skips] 
validation_data = df.iloc[limit::skips]

In [7]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [8]:
scaler.fit(train_data)
scaled_train = scaler.transform(train_data)
scaled_test = scaler.transform(validation_data)

In [9]:
from keras.preprocessing.sequence import TimeseriesGenerator

In [10]:
# define generator
n_input = 11 * 12 # 11 years 
n_features = 1 # we are only using 1 timeseries to make our predictions
generator = TimeseriesGenerator(scaled_train, scaled_train, length=n_input, batch_size=1)

In [11]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [12]:
# define model
model = Sequential() # adds layers in a sequence
model.add(LSTM(100, activation='relu', return_sequence=True, input_shape=(n_input, n_features))) # maybe try elu?
# 300 neurons, return_sequences=True to add more LSTM layers
model.add(LSTM(100, activation='relu')) # 100 neurons
#model.add(LSTM(75, activation='relu')) # 75 neurons
#model.add(Dense(50, activation='relu')) # additional Dense layer with 50 neurons
model.add(Dense(1))
# 1 layer
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='mse')
# smaller learning rate = longer running time
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50)                10400     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 10,451
Trainable params: 10,451
Non-trainable params: 0
_________________________________________________________________


In [13]:
# fit model
epochs = 100

# Define a ModelCheckpoint callback to save the model weights based on validation loss
#checkpoint = ModelCheckpoint('best_weights.h5', monitor='val_loss', save_best_only=True, save_weights_only=True)

# Fit the model to the data, using the ModelCheckpoint callback
history = model.fit(generator, epochs=epochs) # validation_data=(scaled_test,scaled_test) callbacks=[checkpoint])

Epoch 1/100
2190/2190 [==============================] - 38s 17ms/step - loss: 0.1545
Epoch 2/100
 743/2190 [=========>....................] - ETA: 24s - loss: 0.0814

KeyboardInterrupt: 

In [ ]:
loss_per_epoch = model.history.history['loss']
#val_loss_per_epoch = model.history.history['val_loss']
plt.plot(range(len(loss_per_epoch)),loss_per_epoch);
#plt.plot(range(len(val_loss_per_epoch)), val_loss_per_epoch)

In [ ]:
last_train_batch = scaled_train[-n_input:] 
# take the last n_input month of values, to make 
# predictions on the 1st test set

In [ ]:
last_train_batch = last_train_batch.reshape((1, n_input, n_features))

In [ ]:
#model.predict(last_train_batch)

y_pred = model.predict(generator)

st = scaled_train.shape[0]
yp = y_pred.shape[0]
c = st - yp
# Note: predictions start after 'n_input' numbers 

In [ ]:
plt.plot(np.arange(c, st), y_pred, label='predict') 
plt.plot(scaled_train, label='train')
plt.legend();

#### Comment on plot:

The above plot shows how well our model fits with the training data. As the orange and blue lines almost completely overlap, we can conclude that the model accurately fits with the data. This is a necessary check, as our model will have no chance of predicting future sunspot numbers if it cannot successfully predict the training values.

In [ ]:
validation_predictions = []

first_eval_batch = scaled_train[-n_input:]
current_batch = first_eval_batch.reshape((1, n_input, n_features))

for i in range(len(validation_data)):
    
    # get the prediction value for the first batch
    current_pred = model.predict(current_batch)[0]
    
    # append the prediction into the array
    validation_predictions.append(current_pred) 
    
    # use the prediction to update the batch and remove the first value
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

In [ ]:
validation_predictions
# NOTE: these predictions are in the range 0-1
# we need to convert it back 

In [ ]:
true_predictions = scaler.inverse_transform(validation_predictions)

In [ ]:
validation_data['Predictions'] = true_predictions

In [ ]:
validation_data.plot(figsize=(14,5));

#### Comment on plot:

The predictions for future sunspot number seem to have a period which is close to the observed period (but not perfect), and the amplitude is still too high. I can try running again with more epochs or more neurons and see what happens.

**Edit**: run with 300 neurons produced a similar result to one with 200 neurons. Next I will try running with 200 neurons again, but for 100 epochs.

**Edit 2**: run with 200 neurons for 87 epochs produced results with a smaller amplitude, but the timing of the peaks were incorrect.

**Edit 3**: run with 300 neurons for 78 epochs --> fitting not appropriate, previous attempt was the best.

**Edit 4**: run with 100 neurons in layer 1, 100 neurons in layer 2, for 100 epochs --> also predicting for a longer period. LSTM does not perform too badly, however it has a problem predicting how low the sunspot numbers go and the amplitudes.